In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

import pymysql
import requests
import json
from datetime import datetime
import time
import logging

In [3]:
# 데이터 불러오기 및 필요한 함수 정의
base_src = 'Data'

# user 데이터
research_src = os.path.join(base_src, 'research_data_03.csv')
research = pd.read_csv(research_src,
                    sep=',',
                    encoding = 'cp949')
research = research.set_index('identifier')
research

,TIMESTAMP,1. 직업이 어떻게 되시나요?,2. 출생년도를 알려주세요!,3. 성별을 알려주세요!,4. 지난 한 달 동안 책을 몇 권 읽었나요?,5. 책을 읽는 시간은 하루에 얼마나 되나요?,6. 책을 읽고 내용을 기록하고 싶은 적이 있으셨나요?,7. 책을 읽고 내용에 대해 다른 사람과 이야기하고 싶은 적이 있으셨나요?,8-1. 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요!( 평점 범위 : 0.0 ~ 5.0 ) [책 01],8-1. 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요!( 평점 범위 : 0.0 ~ 5.0 ) [평점 01],...,8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 16],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 17],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 17],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 18],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 18],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 19],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 19],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 20],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 20],9. 기프티콘 추첨을 위해 번호를 입력해주세요!
identifier,,,,,,,,,,,,,,,,,,,,,
research066,2022-09-26 16:07:06,직장인,1997,MALE,한 권도 읽지 않았다.,0시간,1,1,돌이킬 수 없는 약속,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1028590730
research067,2022-09-26 18:20:28,취준생,1994,FEMALE,1권 ~ 2권,1시간,5,5,에세이스트의 책상,3.0,...,3.5,너의 슬픔이 끼어들 떄,3.0,시선으로부터,4.0,i에게,4.0,내가 되는 꿈,3.5,1064295792
research068,2022-09-27 10:35:44,직장인,2000,FEMALE,1권 ~ 2권,1시간,3,4,멸망한 세계에서 살아남는 방법,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1030084647
research069,2022-09-27 11:21:22,학생,1998,FEMALE,3권 ~ 4권,2시간,5,5,차녀힙합,4.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023123462
research070,2022-09-27 11:52:11,취업준비생,1998,FEMALE,한 권도 읽지 않았다.,0시간,5,5,모모,5.0,...,4.0,소년이 온다,5.0,총 균 쇠,4.5,봉제인형 살인사건,3.0,지대넓얕 1,4.0,1089768992
research071,2022-09-27 14:03:02,직장인,1997,MALE,1권 ~ 2권,0시간,4,2,나의 첫 투자 수업 1 : 마인드편,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023393630


In [4]:
#   db 연결
def dbconnect():
    conn = pymysql.connect (host='chaekin.cvg3ycl5pbuo.ap-northeast-2.rds.amazonaws.com', user='admin', password='chrlghk1!', db='testdb', charset='utf8')
    return conn

conn = dbconnect()  #   DB 연결

In [5]:
'''
member TB에서 identifier에 해당하는 row의 id를 가져옴
'''
def find_member_id(conn, identifier):
    cur = conn.cursor()
    sql = "SELECT id FROM member WHERE identifier = %s"
    cur.execute(sql, identifier)
    result = cur.fetchall()
    conn.commit()
    return result

In [6]:
'''
member TB에 member insert
'''
def insert_member(conn, member):
    cursor = conn.cursor()

    sql = "INSERT INTO member (age, gender, identifier, job, nickname) VALUES (%s, %s, %s, %s, %s)"
    
    cursor.execute(sql, member)
    conn.commit()

In [7]:
'''
research TB에 member_id, title, score insert
'''
def insert_research(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO research (member_id, title, score) VALUES (%s, %s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()

In [8]:
'''
research TB에서 모든 record 가져오기
'''
def get_research_records(conn):
    cur = conn.cursor()
    sql = "SELECT * FROM research"
    cur.execute(sql)
    result = cur.fetchall()
    conn.commit()
    return result

In [28]:
'''
research TB에서 id가 n 이상인 record 가져오기
'''
def get_n_research_records(conn, n):
    cur = conn.cursor()
    sql = "SELECT * FROM research where id >= %s"
    cur.execute(sql, n)
    result = cur.fetchall()
    conn.commit()
    return result

In [9]:
'''
research TB에 isbn update
'''
def update_isbn(conn, item):
    cur = conn.cursor()
    sql = "update research set isbn = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [10]:
'''
research TB에 book_isbn_id update
'''
def update_book_isbn_id(conn, item):
    cur = conn.cursor()
    sql = "update research set book_isbn_id = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [11]:
'''
book_isbn TB에서 isbn에 해당하는 book_isbn_id 가져오기
'''
def get_book_isbn_id(conn, isbn):
    cur = conn.cursor()
    sql = "SELECT * FROM book_isbn where isbn = %s"
    cur.execute(sql,isbn)
    result = cur.fetchall()
    conn.commit()
    return result

In [25]:
'''
book_isbn TB에 isbn insert
'''
def insert_book_isbn(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO book_isbn (id, isbn) VALUES (%s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()

In [13]:
'''
research TB에서 모든 record 가져오기
'''
def search_title(title):
     #   요청 api 주소
    API_HEAD = 'http://www.aladin.co.kr/ttb/api/ItemSearch.aspx?ttbkey='
    #   api key
    API_KEYS = 'ttbbeckhem961036001'
    API_BODY = '&Query='
    API_TAIL = '&QueryType=Title&MaxResults=1&start=1&SearchTarget=Book&output=js&Version=20131101'

    url = API_HEAD + API_KEYS + API_BODY + title + API_TAIL
    res = requests.get(url)
    item = res.json()['item']
    if item:
        isbn = item[0]['isbn13']
    else:
        isbn = None
    
    return isbn

In [26]:
'''
booklog TB에 book_id, member_id, read_status insert
'''
def insert_booklog(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO booklog (book_id, member_id, read_status) VALUES (%s, %s, 'COMPLETE')"
    
    cursor.execute(sql, item)
    conn.commit()
    
    cursor = conn.cursor()

    sql = "select last_insert_id() from booklog"
    
    booklog_id = cursor.execute(sql)
    conn.commit()
    
    return booklog_id

In [15]:
'''
research TB에 booklog_id update
'''
def update_booklog_id(conn, item):
    cur = conn.cursor()
    sql = "update research set booklog_id = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [16]:
'''
review TB에 booklog_id, score insert 후 book score, count update
'''
def insert_review(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO review (booklog_id, score) VALUES (%s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()
    
    cursor = conn.cursor()

    sql = "select last_insert_id() from review"
    
    review_id = cursor.execute(sql)
    conn.commit()

    cur = conn.cursor()
    sql = "SELECT * FROM book WHERE id = (SELECT book_id FROM booklog WHERE id = %s);"
    cur.execute(sql, item[0])
    book = cur.fetchall()[0]
    conn.commit()
    
    score = book[10]
    count = book[11]
    
    update_count = count + 1
    update_score = (score * count + item[1])/update_count
    
    item2 = (update_score, update_count, book[0])
    
    cur = conn.cursor()
    sql = "update book set rating_score = %s, rating_count = %s where id = %s"
    cur.execute(sql, item2)
    conn.commit()
    
    return review_id

In [17]:
'''
research TB에 review_id update
'''
def update_review_id(conn, item):
    cur = conn.cursor()
    sql = "update research set review_id = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [18]:
# 회원 정보 insert
for row in research.itertuples():
    data = (row[3], row[4], row[0], row[2], row[0])
    if len(find_member_id(conn, row[0])) == 0:
        insert_member(conn, data)
        print(data)

(1997, 'MALE', 'research066', '직장인', 'research066')
(1994, 'FEMALE', 'research067', '취준생', 'research067')
(2000, 'FEMALE', 'research068', '직장인', 'research068')
(1998, 'FEMALE', 'research069', '학생', 'research069')
(1998, 'FEMALE', 'research070', '취업준비생', 'research070')
(1997, 'MALE', 'research071', '직장인', 'research071')


In [19]:
# research TB에 member_id, title, score insert
for row in research.itertuples():
    member_id = find_member_id(conn, row[0])[0][0]
    print(member_id)
    for i in (range(9, 48, 2)):
        if pd.isna(row[i]):
            break
        print(row[i]," ",row[i+1])
        item = (member_id, row[i], row[i+1])
        insert_research(conn, item)

66
돌이킬 수 없는 약속   4.0
화차   5.0
사피엔스   4.0
67
에세이스트의 책상   3.0
이제야 언니에게   3.5
기분을 관리하면 당신도 잘 살 수 있습니다   3.5
살고 싶다는 농담   4.0
일하고 일하고 사랑을 하고   3.0
2022 제13회 젊은 작가상 수상 작품집   3.5
쓸쓸해서 머나먼   3.0
바다는 잘 있습니다   4.0
편의점   3.0
낙하하는 저녁   3.5
물고기는 존재하지 않는다   5.0
복자에게   4.0
동급생   3.5
여름은 오래 그곳에 남아   3.0
자기 앞의 생   3.0
피라네시   3.5
너의 슬픔이 끼어들 떄   3.0
시선으로부터   4.0
i에게   4.0
내가 되는 꿈   3.5
68
멸망한 세계에서 살아남는 방법   5.0
버그를 악용하지 마세요   4.0
파반느   4.0
세비지 포레스트   4.5
내 벽을 움킨 해일   3.5
나세 엘토라   4.0
내가 m이라니   5.0
이오타의 샘   3.8
아폴리티카   5.0
디어 에데르트   5.0
그녀와 야수   3.5
부서진 세계의 구원자   3.8
나를 파멸시킨 그대에게   4.5
태양의 주인   4.5
69
차녀힙합   4.9
아가미   3.5
최소한의 선의   4.2
따님이 기가 세요   4.2
대스타   4.7
목소리를 드릴게요   4.9
70
모모   5.0
밤의 이야기꾼들   3.0
시선으로부터   5.0
달러구트 꿈 백화점 1   4.0
나는 소망한다 내게 금지된 것을   5.0
걸어다니는 어원 사전   3.0
아침에는 죽음을 생각하는 것이 좋다   4.0
황금털 사자   4.0
희랍어시간   5.0
콰이어트   4.0
넛지   3.0
나인   5.0
정의란 무엇인가   4.5
멋진 신세계   4.0
우리가 빛의 속도로 갈 수 없다면   3.5
거꾸로 읽는 세계사   4.0
소년이 온다   5.0
총 균 쇠   4.5
봉제인형 살인사건   3.0
지대넓얕 1   4.0
71
나의 첫 투자 수업 1 : 마인드편   4.0


In [20]:
# 책 이름으로 검색해서 isbn 받아오기
research_records = get_research_records(conn);
for row in research_records:
    print(row[4] == None)
    if row[4] == None:
        isbn = search_title(row[2])
        item = (isbn, row[0])
        if item:
            update_isbn(conn, item)
        print(item, row[2])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [27]:
# book_isbn에 추가
research_records = get_research_records(conn)
index = 176
for row in research_records:
    if row[5] != None:
        continue
    print(row)
    isbn = row[4]
    book_isbn_id = get_book_isbn_id(conn, isbn)
    if len(book_isbn_id) == 0:
        item = (index, isbn)
        insert_book_isbn(conn, item)
        book_isbn_id = get_book_isbn_id(conn, isbn)
        index = index + 1
    
    item2 = (book_isbn_id[0][0], row[0])
    print(item2)
    update_book_isbn_id(conn, item2)

(627, 67, '기분을 관리하면 당신도 잘 살 수 있습니다', 3.5, '9791187147961', None, None, None)
(176, 627)
(628, 67, '살고 싶다는 농담', 4.0, '9788901244600', None, None, None)
(5735384, 628)
(629, 67, '일하고 일하고 사랑을 하고', 3.0, '9788936424725', None, None, None)
(177, 629)
(630, 67, '2022 제13회 젊은 작가상 수상 작품집', 3.5, '9788954685887', None, None, None)
(178, 630)
(631, 67, '쓸쓸해서 머나먼', 3.0, '9788932020303', None, None, None)
(1039092, 631)
(632, 67, '바다는 잘 있습니다', 4.0, '9788932030395', None, None, None)
(3625234, 632)
(633, 67, '편의점', 3.0, '9791161571379', None, None, None)
(2, 633)
(634, 67, '낙하하는 저녁', 3.5, '9791160270259', None, None, None)
(179, 634)
(635, 67, '물고기는 존재하지 않는다', 5.0, '9791189327156', None, None, None)
(63, 635)
(636, 67, '복자에게', 4.0, '9788954674492', None, None, None)
(5744698, 636)
(637, 67, '동급생', 3.5, '9791138413541', None, None, None)
(180, 637)
(638, 67, '여름은 오래 그곳에 남아', 3.0, '9788934972204', None, None, None)
(3287419, 638)
(639, 67, '자기 앞의 생', 3.0, '9788982816635', None, None, None)
(279356, 639

In [30]:
# booklog에 추가
n = 622
research_records = get_n_research_records(conn,n)
for row in research_records:
    print(row)
    if row[6] == None:
        item = (row[5], row[1])
        print(item)
        try:
            booklog_id = insert_booklog(conn, item)
            item2 = (booklog_id, row[0])
            update_booklog_id(conn, item2)
        except:
            print("error " , item2)

(622, 66, '돌이킬 수 없는 약속', 4.0, '9788998274795', 3411733, None, None)
(3411733, 66)
error  (5860446, 685)
(623, 66, '화차', 5.0, '9788954617437', 1235510, None, None)
(1235510, 66)
(624, 66, '사피엔스', 4.0, '9788934991328', 5818691, None, None)
(5818691, 66)
(625, 67, '에세이스트의 책상', 3.0, '9788954680400', 6040857, None, None)
(6040857, 67)
error  (419, 624)
(626, 67, '이제야 언니에게', 3.5, '9788936438012', 5480229, None, None)
(5480229, 67)
(627, 67, '기분을 관리하면 당신도 잘 살 수 있습니다', 3.5, '9791187147961', 176, None, None)
(176, 67)
(628, 67, '살고 싶다는 농담', 4.0, '9788901244600', 5735384, None, None)
(5735384, 67)
(629, 67, '일하고 일하고 사랑을 하고', 3.0, '9788936424725', 177, None, None)
(177, 67)
(630, 67, '2022 제13회 젊은 작가상 수상 작품집', 3.5, '9788954685887', 178, None, None)
(178, 67)
(631, 67, '쓸쓸해서 머나먼', 3.0, '9788932020303', 1039092, None, None)
(1039092, 67)
(632, 67, '바다는 잘 있습니다', 4.0, '9788932030395', 3625234, None, None)
(3625234, 67)
error  (425, 631)
(633, 67, '편의점', 3.0, '9791161571379', 2, None, None)
(2, 67)
(6

In [32]:
# review에 추가 & book score, count update
n = 622
research_records = get_n_research_records(conn,n)
for row in research_records:
    print(row[6])
    if row[6] != None and row[7] == None:
        item = (row[6], row[3])
        review_id = insert_review(conn, item)
        item2 = (review_id, row[0])
        update_review_id(conn, item2)
        print(item, item2)

None
625
(625, 5.0) (623, 418)
626
(626, 4.0) (624, 419)
None
628
(628, 3.5) (626, 420)
629
(629, 3.5) (627, 421)
630
(630, 4.0) (628, 422)
631
(631, 3.0) (629, 423)
632
(632, 3.5) (630, 424)
633
(633, 3.0) (631, 425)
None
635
(635, 3.0) (633, 426)
636
(636, 3.5) (634, 427)
637
(637, 5.0) (635, 428)
638
(638, 4.0) (636, 429)
639
(639, 3.5) (637, 430)
None
641
(641, 3.0) (639, 431)
642
(642, 3.5) (640, 432)
None
644
(644, 4.0) (642, 433)
None
None
647
(647, 4.0) (647, 434)
648
(648, 3.5) (655, 435)
649
(649, 4.5) (658, 436)
650
(650, 4.9) (659, 437)
None
652
(652, 4.2) (661, 438)
None
654
(654, 4.7) (663, 439)
655
(655, 4.9) (664, 440)
656
(656, 5.0) (665, 441)
None
658
(658, 5.0) (667, 442)
659
(659, 4.0) (668, 443)
None
661
(661, 3.0) (670, 444)
None
663
(663, 4.0) (672, 445)
664
(664, 5.0) (673, 446)
None
666
(666, 3.0) (675, 447)
None
None
None
None
None
672
(672, 5.0) (681, 448)
673
(673, 4.5) (682, 449)
None
675
(675, 4.0) (684, 450)
None


In [ ]:
update research set booklog_id = 
(select booklog.id from booklog where booklog.member_id = research.member_id and booklog.book_id = research.book_isbn_id) 
where research.id >= 622 and research.booklog_id is not NULL;